In [142]:
%pip install ankipandas numpy plotly kaleido


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [143]:
%pip install --upgrade nbformat


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [144]:
# Imports
import pandas as pd
import re
import random
import copy
import math
import json
import plotly.graph_objects as go
import numpy as np
import ankipandas
from collections import Counter

# Load Shared Datasets

This section of code loads the [Google N-Grams Data](https://github.com/orgtre/google-books-ngram-frequency/blob/main/ngrams/1grams_english.csv), along with `ipadict` for [phonetic information](https://github.com/open-dict-data/ipa-dict/blob/master/data/en_US.txt).

In [145]:
# Load in the one-gram information (word frequencies)
one_grams = pd.read_csv("1grams_english.csv")
one_grams_dict = {row[1]['ngram'].lower(): row[1]['freq'] for row in one_grams.iterrows()}

# Scoring methods designed to select one of the options from the IPA dictionary.
# It selects the shortest words with the most schwas (laziest pronunciation).


def score(word):
    word.replace(" ", "")
    return (len(word), -word.count("ə"))


def get_laziest(ipa_list):
    return list(sorted(ipa_list, key=score))[0]


# Loading the dictionary and parsing out the best scoring pronunciation
ipa = pd.read_csv("en_US.txt", sep='\t', names=["eng", "ipa"])
ipa_opt_dict = {row[1]['eng']: row[1]['ipa'].split(', ') for row in ipa.iterrows()}
ipa_dict = {row[1]['eng']: get_laziest(row[1]['ipa'].split(', ')) for row in ipa.iterrows()}

# Write Common Code

This is code that takes in:

1. A list of words,
2. A dictionary of word transformations,
3. An optional function that computes lengths of words (defaults to `len`),
4. An optional function that transforms a transformed word into the constituent "letters".

It then returns the various statistics of that model as a dictionary:
1. The mean length normalized to assume a binary alphabet (multiplied by the $log_2$ of the alphabet size),
2. The reconstruction entropy $H(X|f(X))$,
3. The reconstruction error $\mathbf{P}\{X = \mathrm{argmax}_{y : f(y) =  f(X)} p(y)\}$

In [146]:
def entropy(counts):
    entropy = 0
    total = 0
    for x in counts:
        total += counts[x]
    for x in counts:
        entropy -= (counts[x]/total)*math.log2(counts[x]/total)
    
    return entropy

def transmission_cost(letters,suprisal):
    entropy = 0
    for l in letters:
        entropy += suprisal[l]
    
    return entropy

def entropy_wl(word_list):
    countlist = {w:one_grams_dict[w] for w in word_list}
    return entropy(countlist)

def code_statistics(word_list, word_mapping, length_function = lambda x: len(x), letter_making = lambda x: x, delim = None):
    # Data to track mean outline length
    denom = 0
    total_length = 0

    # Data to track alphabet size
    letter_counts = Counter()

    # Data to track for computing reconstruction entropy and reconstruction probability
    forward = {}
    inverse = {"": {"": float('inf')}}
    most_probable = {}
    totals = {"": float('inf')}

    # iterate over word list keeping track of stats for mean length and the inverse mapping
    for word in word_list:
        count = one_grams_dict[word]

        image = word_mapping[word]
        if delim:
            image += delim

        word_length = length_function(image) * count
        total_length += word_length
        denom += count

        forward[word] = image
        if image not in inverse:
            inverse[image] = {}
            most_probable[image] = word
        inverse[image][word] = count+1
        totals[image] = totals.get(image, 1) + count

        current_letters = letter_making(image)
        letter_counts.update(current_letters)

    letter_total = letter_counts.total()
    suprisal = {letter: -math.log2(letter_counts[letter]/letter_total) for letter in letter_counts}

    # compute the reconstruction entropy and reconstruction probability
    total_reconstruction_entropy = 0
    total_reconstruction_probability = 0
    total_transmission_cost = 0
    for word in word_list:
        count = one_grams_dict[word]
        total_reconstruction_entropy -= float('-inf') if inverse[forward[word]][word]/totals[forward[word]] == 0 else count * math.log2(inverse[forward[word]][word]/totals[forward[word]])
        total_reconstruction_probability += count if word == most_probable[forward[word]] else 0
        total_transmission_cost += count*transmission_cost(letter_making(forward[word]), suprisal)

    running_lengths = 0
    running_count = 1

    for w in one_grams_dict:
        if w not in word_list:
            continue
        prob = one_grams_dict[w]/denom
        running_count += 1
        running_lengths += prob*math.log2(running_count)

    return {"mean_binary_length": math.log2(len(letter_counts))*total_length / denom,
            "mean_binary_excess": math.log2(len(letter_counts))*total_length / denom - running_lengths,
            "mean_entropy_shift": math.log2(len(letter_counts))*total_length / denom - entropy_wl(word_list),
            "mean_transmission_cost": total_transmission_cost / denom,
            "mean_transmission_excess": total_transmission_cost / denom - running_lengths,
            "mean_transmission_shift": total_transmission_cost / denom - entropy_wl(word_list),
            "reconstruction_entropy": total_reconstruction_entropy / denom,
            "reconstruction_error": (1-total_reconstruction_probability / denom),
            "alphabet": " ".join([x for x, _ in letter_counts.most_common()])}

# Pure Spelling

In [147]:
spelling = {x: x for x in one_grams_dict}

In [148]:
code_statistics([x for x in one_grams_dict if x in spelling], spelling)

{'mean_binary_length': 21.740674602773286,
 'mean_binary_excess': 15.093721783000312,
 'mean_entropy_shift': 11.973528232304314,
 'mean_transmission_cost': 19.08533447185226,
 'mean_transmission_excess': 12.438381652079286,
 'mean_transmission_shift': 9.318188101383289,
 'reconstruction_entropy': 0.0,
 'reconstruction_error': 0.0,
 'alphabet': "e i s t a n r o l c d p u g m h y b f v w k x z q j ' é"}

# Pure IPA representation
This removes some of the noise from the IPA representation to allow it to be tested as an encoding.

In [149]:
reduced_ipa = {x: ipa_dict[x].replace("/","").replace("ˌ","").replace("ˈ","") for x in ipa_dict}

In [150]:
code_statistics([x for x in one_grams_dict if x in reduced_ipa], reduced_ipa)

{'mean_binary_length': 20.47519327609292,
 'mean_binary_excess': 13.829919120320977,
 'mean_entropy_shift': 10.710090184552453,
 'mean_transmission_cost': 19.268881294853912,
 'mean_transmission_excess': 12.623607139081969,
 'mean_transmission_shift': 9.503778203313445,
 'reconstruction_entropy': 0.042481294979739674,
 'reconstruction_error': 0.014930232236564533,
 'alphabet': 'ə ɪ t n s ɹ d ɫ k i ɛ p z m ɝ æ ʃ e a ɑ ʊ f b ŋ v ɔ o ɡ ʒ u w j h θ ð'}

# Load Carter Briefwords
The dictionary is available [here](https://www.reddit.com/r/shorthand/comments/xg7k10/a_briefhand_resource/).

In [151]:
briefhand_raw = pd.read_csv("dict.csv")

briefhand = {}

for _,line in briefhand_raw.iterrows():
    briefhand[line['word']] = line['brief']

In [152]:
code_statistics([x for x in one_grams_dict if x in briefhand], briefhand)

{'mean_binary_length': 8.65318533787457,
 'mean_binary_excess': 3.69443099723304,
 'mean_entropy_shift': 0.9394725601340319,
 'mean_transmission_cost': 7.763512590151817,
 'mean_transmission_excess': 2.8047582495102867,
 'mean_transmission_shift': 0.04979981241127884,
 'reconstruction_entropy': 1.0613532563363244,
 'reconstruction_error': 0.3048400643277738,
 'alphabet': "s r t n l c d a p m g o f b v i u e k w h y x z q j 1 0 2 3 5 % 7 '"}

# Dutton Speedwords
The dictionary is available [here](http://www2.cmp.uea.ac.uk/~jrk/conlang.dir/Speedwords.dict).

In [153]:
with open("Speedwords.dict", "r") as file:
    speedwords_raw = file.readlines()

speedwords = {}

for line in speedwords_raw[89:]:
    if line == '\n':
        continue

    splits = line.replace("\n","").split("\t")
    if len(splits) != 2:
        continue

    rep = splits[0]
    words = splits[1]
    word_list = words.split(",")
    for word in word_list:
        speedwords[re.sub(r'[\(\[{].*?[\)\]}]', '', word)] = re.sub(r'[\(\[{].*?[\)\]}]', '', rep.replace("-",""))

In [154]:
code_statistics([x for x in one_grams_dict if x in speedwords], speedwords)

{'mean_binary_length': 9.329723457179032,
 'mean_binary_excess': 4.124412180082545,
 'mean_entropy_shift': 1.2843567331541461,
 'mean_transmission_cost': 9.034227700629268,
 'mean_transmission_excess': 3.828916423532781,
 'mean_transmission_shift': 0.9888609766043821,
 'reconstruction_entropy': 0.38624091801605515,
 'reconstruction_error': 0.11017558339765876,
 'alphabet': 'e o a i u r s t l n k p m g v f b d y x c z h j q w & O K'}

# Gregg Anniversary

This loads in Gregg Anniversary from [here](https://github.com/grascii/dictionaries).  The key here is that blends should be counted as single alphabet entries, and all is separated by hyphens, so we need custom length and letter functions.

In [155]:
gregg_anniversary_raw = pd.read_csv("anniversary_core.csv")
gregg_anniversary_raw_2= pd.read_csv("anniversary_supplement.csv")

gregg_anniversary = {}

for _,line in gregg_anniversary_raw.iterrows():
    gregg_anniversary[str(line['word']).lower()] = re.sub("^/^-","",line['form'])

for _,line in gregg_anniversary_raw_2.iterrows():
    gregg_anniversary[str(line['word']).lower()] = re.sub("^/^-","",line['form'])

gregg_length = lambda x: x.count("-")+1
gregg_letters = lambda x: x.split("-")

In [156]:
code_statistics([x for x in one_grams_dict if x in gregg_anniversary], gregg_anniversary, length_function = gregg_length, letter_making = gregg_letters)

{'mean_binary_length': 15.425163710803988,
 'mean_binary_excess': 9.260242654630629,
 'mean_entropy_shift': 6.228619368943665,
 'mean_transmission_cost': 11.685583747096876,
 'mean_transmission_excess': 5.520662690923517,
 'mean_transmission_shift': 2.4890394052365536,
 'reconstruction_entropy': 0.25155755894450227,
 'reconstruction_error': 0.07505138574335446,
 'alphabet': 'e s a r k t n l d o s2 u m \\ p sh h f v / i b pr g j ^ nt e2 tn th mn ch pl ss tm df td a2 br ng bl fr ea nk ld ya x jnt fl th2 w ss2 i2 ia mt ye kp ye2 o2 dfl c s2h s2s z bld ya2 nd us E  q ao oe > vr s22 u\\ ea2 ot dm mm dya ths y as2 s2s2 < xs \\v se rd o1 re om s2o es pld nu ge ta rdf'}

# Gregg Notehand

This loads in Gregg Notehand from [here](https://gregg-shorthand.com/2015/09/05/notehand-dictionary/).  The key here is that blends should be counted as single alphabet entries, and all is separated by hyphens, so we need custom length and letter functions.

In [157]:
gregg_notehand_raw = pd.read_csv("Notehand.csv")

gregg_notehand = {}

for _,line in gregg_notehand_raw.iterrows():
    gregg_notehand[str(line['word']).lower()] = str(line['form'])

In [158]:
code_statistics([x for x in one_grams_dict if x in gregg_notehand], gregg_notehand, length_function = gregg_length, letter_making = gregg_letters)

{'mean_binary_length': 15.343301937676637,
 'mean_binary_excess': 9.916704355750431,
 'mean_entropy_shift': 7.050406666597668,
 'mean_transmission_cost': 12.386150634947892,
 'mean_transmission_excess': 6.959553053021686,
 'mean_transmission_shift': 4.0932553638689235,
 'reconstruction_entropy': 0.11181820004163713,
 'reconstruction_error': 0.033954582356027774,
 'alphabet': 'e r a t s l left_s k o oo p n d m f b v I sh ing g h j  nt nd ten ch ted rd under_th over_th ld ng men oi ia _ ngk md Ia th nan ye ea c mt . e_p ya I_rd over_the x r–s left d–s e_d over nk i er a_ ee + e_t a_sh'}

# Gregg Simplified

This is a different encoding of Gregg simplified, from [here](https://www.reddit.com/r/shorthand/comments/1e7ie7g/gregg_simplified_computerreadable_dictionary/)

In [159]:
gregg_simplified_raw = pd.read_csv("LEGS.csv",names=['word','form'],header = None)

gregg_simplified = {}

for _,line in gregg_simplified_raw.iterrows():
    gregg_simplified[str(line['word']).lower()] = str(line['form'])

In [160]:
code_statistics([x for x in one_grams_dict if x in gregg_simplified], gregg_simplified, letter_making = lambda x: x)

{'mean_binary_length': 11.769702509512225,
 'mean_binary_excess': 7.344900086927768,
 'mean_entropy_shift': 4.669998288575941,
 'mean_transmission_cost': 10.815613232997505,
 'mean_transmission_excess': 6.390810810413048,
 'mean_transmission_shift': 3.7159090120612204,
 'reconstruction_entropy': 0.2091399622585796,
 'reconstruction_error': 0.06600137338137413,
 'alphabet': 'e r a t l o k n s z p u m d N b v i T f $ D h g j : M c ( 5 q G R ) L w V y K X ^ 3 % J x A ~ P *'}

# bref

This loads the bref dictionary from [here](https://www.reddit.com/r/shorthand/comments/esjhdk/bref_shorthand/).

In [161]:
with open("11661 WORDS FORWARD.txt", "r") as file:
    bref_raw = file.readlines()

bref = {}

for line in bref_raw:
    splits = line.split(" = ")
    if len(splits) != 2:
        continue
    rep = splits[1].replace("\n","")
    word = splits[0]
    bref[word] = rep

In [162]:
code_statistics([x for x in one_grams_dict if x in bref], bref)

{'mean_binary_length': 12.321204615957537,
 'mean_binary_excess': 5.900270582891406,
 'mean_entropy_shift': 2.8401015434801753,
 'mean_transmission_cost': 11.515170421469378,
 'mean_transmission_excess': 5.094236388403247,
 'mean_transmission_shift': 2.034067348992016,
 'reconstruction_entropy': 0.0,
 'reconstruction_error': 0.0,
 'alphabet': 's r t n c d l p a m g o e f i b u z v w k j h x y q'}

# Yublin

This loads the Yublin dictionary from [here](http://jonathanaquino.com/yublin.csv).

In [163]:
with open("yublin.csv", "r") as file:
    raw_yublin = file.readlines()

yublin = {}

for line in raw_yublin:
    splits = line.replace("\n","").split(",")
    if len(splits) != 2:
        continue
    rep = splits[1]
    word = splits[0]

    yublin[word] = rep

for x in one_grams_dict:
    if x not in yublin:
        yublin[x] = x

In [164]:
code_statistics([x for x in one_grams_dict if x in yublin], yublin)

{'mean_binary_length': 16.559197673153815,
 'mean_binary_excess': 9.912244853380841,
 'mean_entropy_shift': 6.792051302684843,
 'mean_transmission_cost': 14.849250768890093,
 'mean_transmission_excess': 8.202297949117119,
 'mean_transmission_shift': 5.082104398421121,
 'reconstruction_entropy': 0.0,
 'reconstruction_error': 0.0,
 'alphabet': "e i s t a n r o l c d p u g m h y b f v w k x z j q ' é"}

# Cut Spelng

This loads the Cut Spelling dictionary form [here](https://github.com/DanielTillett/CutSpel/blob/master/cutspel.csv).

In [165]:
cut_spelng_raw = pd.read_csv("cutspel.csv",names = ["word","brief"], header = None)

cut_spelng = {}

for _,line in cut_spelng_raw.iterrows():
    cut_spelng[line['word']] = line['brief'].replace(" ","")
cut_spelng['muscle'] = 'musl'

for x in one_grams_dict:
    if x not in cut_spelng:
        cut_spelng[x] = x

In [166]:
code_statistics([x for x in one_grams_dict if x in cut_spelng], cut_spelng)

{'mean_binary_length': 19.852082039357892,
 'mean_binary_excess': 13.205129219584919,
 'mean_entropy_shift': 10.084935668888921,
 'mean_transmission_cost': 17.631308792481388,
 'mean_transmission_excess': 10.984355972708414,
 'mean_transmission_shift': 7.864162422012416,
 'reconstruction_entropy': 0.015095746507235148,
 'reconstruction_error': 0.005325956756667138,
 'alphabet': "e i s t n r a o l c d u p m g h y f b v w k j x q z é '"}

# Taylor

This attempts to construct an allignment of the spelling and of the IPA sounds then construct the Taylor representation of that word.

In [167]:
# Improved drop vowels function to handle stress and long vowels and ensure proper vowel removal
def drop_vowels(word):
    vowels = "aeiouAEIOUɑæɪɔʊəɜɛːɒʌ"
    punctuation_and_spaces = "ˈˌ. ,-'\";:!?()[]{}<>/@#%^&*~`"
    return ''.join([char for char in word if char not in vowels and char not in punctuation_and_spaces])

# Recursive alignment function with debug statements including match identification
def find_allowable_pairs_recursive(str1, str2, allowable_pairs, debug = False):
    def helper(s1, s2, pairs, path):
        if not s1 and not s2:
            return path
        for pair in pairs:
            l1, l2 = len(pair[0]), len(pair[1])
            if s1[:l1] == pair[0] and s2[:l2] == pair[1]:
                if debug:
                    print(f"Matching: {pair} with {s1[:l1]} and {s2[:l2]}")
                result = helper(s1[l1:], s2[l2:], pairs, path + [pair])
                if result is not None:
                    return result
        if debug:
            print(f"Failed to match: {s1} with {s2}")
        return None

    return helper(str1, str2, allowable_pairs, [])

# Function to process the word and find alignment with debug statements
def process_word(word, ipa_dict, consonant_pairs, debug = False):
    if word not in ipa_dict:
        return f"Word '{word}' not found in IPA dictionary."
    
    ipa_representation = ipa_dict[word]
    word_consonants = drop_vowels(word)
    ipa_consonants = drop_vowels(ipa_representation)
    
    if debug:
        print(f"Processing '{word}' -> '{word_consonants}' with IPA '{ipa_representation}' -> '{ipa_consonants}'")
    
    alignment = find_allowable_pairs_recursive(word_consonants, ipa_consonants, consonant_pairs, debug = debug)
    
    if alignment is None:
        return f"No valid alignment found for '{word}'."
    return alignment

# Reordered consonant_pairs to prioritize more common matches first and adding back the "f" matching to itself
consonant_pairs = [
    ("c", "k"),  # voiceless velar plosive, usually in "cat"
    ("t", "t"),  # voiceless alveolar plosive
    ("d", "d"),  # voiced alveolar plosive
    ("d", "t"),  # voiced alveolar plosive
    ("t", "d"),  # voiced alveolar plosive
    ("n", "n"),  # alveolar nasal
    ("s", "s"),  # voiceless alveolar fricative
    ("l", "l"),  # alveolar lateral approximant
    ("l", "ɫ"),
    ("r", "r"),  # alveolar approximant matching itself
    ("r", "ɹ"),  # alveolar approximant
    ("r", "ɝ"),  # alveolar approximant
    ("m", "m"),  # bilabial nasal
    ("b", "b"),  # voiced bilabial plosive
    ("k", "k"),  # voiceless velar plosive
    ("f", "f"),  # voiceless labiodental fricative
    ("g", "g"),  # voiced velar plosive
    ("g", "ɡ"),
    ("v", "v"),  # voiced labiodental fricative
    ("x", "ks"), # voiceless velar-alveolar fricative, "taxi"
    ("z", "z"),  # voiced alveolar fricative
    ("p", "p"),  # voiceless bilabial plosive
    ("h", "h"),  # voiceless glottal fricative
    ("w", "w"),  # voiced labial-velar approximant
    ("y", "j"),  # voiced palatal approximant
    ("j", "dʒ"), # voiced palato-alveolar affricate, often "j" sound
    ("j", "ʤ"), # voiced palato-alveolar affricate, often "j" sound
    ("j", "ʒ"), # voiced palato-alveolar affricate, often "j" sound
    ("ng", "ŋ"), # voiced velar nasal
    ("n", "ŋ"), # voiced velar nasal
    ("ch", "ʧ"), # voiceless palato-alveolar affricate
    ("ch", "tʃ"), # voiceless palato-alveolar fricative
    ("sh", "tʃ"), # voiceless palato-alveolar fricative
    ("sh", "ʧ"), # voiceless palato-alveolar affricate
    ("sh", "ʃ"), # voiceless palato-alveolar fricative
    ("ch", "ʃ"), # voiceless palato-alveolar fricative
    ("th", "θ"), # voiceless dental fricative
    ("th", "ð"), # voiced dental fricative, as in "this"
    ("t", "tʃ"), # voiceless palato-alveolar fricative
    ("g", "ʒ"),  # voiced palato-alveolar fricative
    ("s", "z"),  # voiced alveolar fricative, usually in "dogs"
    ("s", "ʒ"),  # voiced palato-alveolar fricative, as in "measure"
    ("s", "ʃ"),  # voiceless palato-alveolar fricative, as in "champagne"
    ("z", "ʒ"),  # voiced palato-alveolar fricative, as in "measure"
    ("z", "ʃ"),  # voiceless palato-alveolar fricative, as in "champagne"
    ("c", "ʃ"),  # voiceless palato-alveolar fricative, as in "champagne"
    ("ck", "k"), # voiceless velar plosive, as in "back"
    ("ps", "s"), # "ps" matching "s"
    ("q", "kw"), # voiceless velar and labial-velar approximant, usually in "queen"
    ("b", ""),   # silent b
    ("h", ""),   # silent h
    ("w", ""),   # silent w
    ("y", ""),   # sometimes silent y
    ("f", "v"),  # voiced labiodental fricative
    ("ph", "f"), # voiceless labiodental fricative, usually in "phone"
    ("gh", "f"), # voiceless labiodental fricative, as in "laugh"
    ("gh", ""),  # silent "gh", as in "though"
    ("kn", "n"), # silent "k", as in "knee"
    ("gn", "n"), # silent "g", as in "gnome"
    ("mn", "m"), # "mn" matching "m"
    ("mn", "n"), # "mn" matching "n"
    ("wr", "r"), # silent "w", as in "write" with matching "r"
    ("wh", "w"), # voiced labial-velar approximant, as in "what"
    ("sc", "s"), # "sc" matching "s"
    ("c", "s"),  # voiceless alveolar fricative, usually in "cent"
    ("ld", "d"), # "dd" matching "d"
    ("dd", "d"), # "dd" matching "d"
    ("ll", "l"), # "ll" matching "l"
    ("ll", "ɫ"),
    ("mm", "m"), # "mm" matching "m"
    ("nn", "n"), # "nn" matching "n"
    ("rr", "r"), # "nn" matching "n"
    ("pp", "p"), # "pp" matching "p"
    ("ff", "f"), # "pp" matching "p"
    ("tt", "t"), # "tt" matching "t"
    ("ss", "s"), # "ss" matching "s"
    ("ss", "z"), # "ss" matching "z"
    ("ss", "ʒ"), # "ss" matching "ʒ"
    ("ss", "ʃ"), # "ss" matching "ʃ"
    ("t", "ʒ"),  # "t" matching "ʒ"
    ("t", "ʃ"),  # "t" matching "ʃ"
    ("x", "gz"), # voiced velar-alveolar fricative, "exam"
    ("x", "z"),  # voiced alveolar fricative, "xylophone"
    ("zz", "ts"),# "zz" matching "ts"
    ("z", "ts"), # "z" matching "ts"
    ("g", "dʒ"), # voiced palato-alveolar affricate
    ("g", "ʤ"), # voiced palato-alveolar affricate
    ("gg", "g"), # "gg" matching voiced velar plosive
    ("gg", "ʒ"), # "gg" matching voiced palato-alveolar fricative
    ("gg", "dʒ"),# "gg" matching voiced palato-alveolar affricate
    ("gg", "ʤ"),# "gg" matching voiced palato-alveolar affricate
    ("ch", "k"), # voiceless velar plosive, as in "school"
    ("cc", "k"),  # voiceless velar plosive, usually in "cat"
    ("", "j"),   # silent matching "j"
    ("p",""),    # silent "p" in words like "pnumonia" or "coup"
    ("t",""),    # silent "p" in words like "pnumonia" or "coup"
    ("sl","l"),  # silent "s" like in "island"
    ("ch",""),   # to handle "yacht"
    ("q","k"),   # to handle "queue"
    ("l","r"),   # to handle "colonel"
    ("s",""),    # silent "s" in words like "debris"
    ("ct","t"),  # to handle "indict"
    ("l",""),    # silent "l" in words like "salmon"
    ("dn","n"),  # to handle "wednesday"
    ("z",""),    # to handle "rendezvous"
    ("r",""),    # to handle "dossier"
    ("ç","s"),   # to handle "façade"
    ("gm","m"),  # to handle "rendezvous"
    ("","w"),    # to handle "bourgeois"
    ("sc","ʃ"),  # "concience"
    ("d",""),    # "handkerchief"
    ('x', 'ɡz'),
    ('gg', 'ɡ'),
    ("cq","kw"),
    ("ng","n"),
    ("kd","t"),
    ("nm","m"),
    ("d","dʒ"),
    ("x","kʃ"),
    ("x","k"),
    ("x","ʃ"),
    ("l","ɝ"),
    ("x","ɡʒ"),
    ("s","tʃ")
]

In [168]:
taylor_convert = {
    ("c", "k"):"k",  # voiceless velar plosive, usually in "cat"
    ("t", "t"):"t",  # voiceless alveolar plosive
    ("d", "d"):"d",  # voiced alveolar plosive
    ("d", "t"):"t",  # voiced alveolar plosive
    ("t", "d"):"t",  # voiced alveolar plosive
    ("n", "n"):"n",  # alveolar nasal
    ("s", "s"):"s",  # voiceless alveolar fricative
    ("l", "l"):"l",  # alveolar lateral approximant
    ("l", "ɫ"):"l",
    ("r", "r"):"r",  # alveolar approximant matching itself
    ("r", "ɹ"):"r",  # alveolar approximant
    ("r", "ɝ"):"r",  # alveolar approximant
    ("m", "m"):"m",  # bilabial nasal
    ("b", "b"):"b",  # voiced bilabial plosive
    ("k", "k"):"k",  # voiceless velar plosive
    ("f", "f"):"f",  # voiceless labiodental fricative
    ("g", "g"):"g",  # voiced velar plosive
    ("g", "ɡ"):"g",
    ("v", "v"):"f",  # voiced labiodental fricative
    ("x", "ks"):"x", # voiceless velar-alveolar fricative, "taxi"
    ("z", "z"):"s",  # voiced alveolar fricative
    ("p", "p"):"p",  # voiceless bilabial plosive
    ("h", "h"):"h",  # voiceless glottal fricative
    ("w", "w"):"w",  # voiced labial-velar approximant
    ("y", "j"):"y",  # voiced palatal approximant
    ("j", "dʒ"):"g", # voiced palato-alveolar affricate, often "j" sound
    ("j", "ʤ"):"g", # voiced palato-alveolar affricate, often "j" sound
    ("j", "ʒ"):"g", # voiced palato-alveolar affricate, often "j" sound
    ("ng", "ŋ"):"ng", # voiced velar nasal
    ("n", "ŋ"):"n", # voiced velar nasal
    ("ch", "ʧ"):"C", # voiceless palato-alveolar affricate
    ("ch", "tʃ"):"C", # voiceless palato-alveolar fricative
    ("sh", "tʃ"):"S", # voiceless palato-alveolar fricative
    ("sh", "ʧ"):"S", # voiceless palato-alveolar affricate
    ("sh", "ʃ"):"S", # voiceless palato-alveolar fricative
    ("ch", "ʃ"):"C", # voiceless palato-alveolar fricative
    ("th", "θ"):"T", # voiceless dental fricative
    ("th", "ð"):"T", # voiced dental fricative, as in "this"
    ("t", "tʃ"):"t", # voiceless palato-alveolar fricative
    ("g", "ʒ"):"g",  # voiced palato-alveolar fricative
    ("s", "z"):"s",  # voiced alveolar fricative, usually in "dogs"
    ("s", "ʒ"):"s",  # voiced palato-alveolar fricative, as in "measure"
    ("z", "ʒ"):"s",  # voiced palato-alveolar fricative, as in "measure"
    ("z", "ʃ"):"s",  # voiceless palato-alveolar fricative, as in "champagne"
    ("s", "ʃ"):"s",  # voiceless palato-alveolar fricative, as in "champagne"
    ("c", "ʃ"):"c",  # voiceless palato-alveolar fricative, as in "champagne"
    ("ck", "k"):"k", # voiceless velar plosive, as in "back"
    ("ps", "s"):"s", # "ps" matching "s"
    ("q", "kw"):"q", # voiceless velar and labial-velar approximant, usually in "queen"
    ("b", ""):"",   # silent b
    ("h", ""):"",   # silent h
    ("w", ""):"",   # silent w
    ("y", ""):"",   # sometimes silent y
    ("f", "v"):"f",  # voiced labiodental fricative
    ("ph", "f"):"f", # voiceless labiodental fricative, usually in "phone"
    ("gh", "f"):"f", # voiceless labiodental fricative, as in "laugh"
    ("gh", ""):"",  # silent "gh", as in "though"
    ("kn", "n"):"n", # silent "k", as in "knee"
    ("gn", "n"):"n", # silent "g", as in "gnome"
    ("mn", "m"):"m", # "mn" matching "m"
    ("mn", "n"):"n", # "mn" matching "n"
    ("wr", "r"):"r", # silent "w", as in "write" with matching "r"
    ("wh", "w"):"w", # voiced labial-velar approximant, as in "what"
    ("sc", "s"):"s", # "sc" matching "s"
    ("c", "s"):"s",  # voiceless alveolar fricative, usually in "cent"
    ("ld", "d"):"d", # "dd" matching "d"
    ("dd", "d"):"d", # "dd" matching "d"
    ("ll", "l"):"l", # "ll" matching "l"
    ("ll", "ɫ"):"l",
    ("mm", "m"):"m", # "mm" matching "m"
    ("nn", "n"):"n", # "nn" matching "n"
    ("rr", "r"):"r", # "nn" matching "n"
    ("pp", "p"):"p", # "pp" matching "p"
    ("ff", "f"):"f", # "pp" matching "p"
    ("tt", "t"):"t", # "tt" matching "t"
    ("ss", "s"):"s", # "ss" matching "s"
    ("ss", "z"):"s", # "ss" matching "z"
    ("ss", "ʒ"):"s", # "ss" matching "ʒ"
    ("ss", "ʃ"):"s", # "ss" matching "ʃ"
    ("t", "ʒ"):"t",  # "t" matching "ʒ"
    ("t", "ʃ"):"t",  # "t" matching "ʃ"
    ("x", "gz"):"x", # voiced velar-alveolar fricative, "exam"
    ("x", "ɡz"):"x", # voiced velar-alveolar fricative, "exam"
    ("x", "z"):"x",  # voiced alveolar fricative, "xylophone"
    ("zz", "ts"):"z",# "zz" matching "ts"
    ("z", "ts"):"z", # "z" matching "ts"
    ("g", "dʒ"):"g", # voiced palato-alveolar affricate
    ("g", "ʤ"):"g", # voiced palato-alveolar affricate
    ("gg", "g"):"g", # "gg" matching voiced velar plosive
    ("gg", "ɡ"):"g",
    ("gg", "ʒ"):"g", # "gg" matching voiced palato-alveolar fricative
    ("gg", "dʒ"):"g",# "gg" matching voiced palato-alveolar affricate
    ("gg", "ʤ"):"g",# "gg" matching voiced palato-alveolar affricate
    ("ch", "k"):"C", # voiceless velar plosive, as in "school"
    ("cc", "k"):"k",  # voiceless velar plosive, usually in "cat"
    ("", "j"):"",   # silent matching "j"
    ("p",""):"",    # silent "p" in words like "pnumonia" or "coup"
    ("t",""):"",    # silent "p" in words like "pnumonia" or "coup"
    ("sl","l"):"l",  # silent "s" like in "island"
    ("ch",""):"",   # to handle "yacht"
    ("q","k"):"k",   # to handle "queue"
    ("l","r"):"r",   # to handle "colonel"
    ("s",""):"",    # silent "s" in words like "debris"
    ("ct","t"):"t",  # to handle "indict"
    ("l",""):"",    # silent "l" in words like "salmon"
    ("dn","n"):"n",  # to handle "wednesday"
    ("z",""):"",    # to handle "rendezvous"
    ("r",""):"",    # to handle "dossier"
    ("ç","s"):"s",   # to handle "façade"
    ("gm","m"):"m",  # to handle "rendezvous"
    ("","w"):"",    # to handle "bourgeois"
    ("sc","ʃ"):"s",  # "concience"
    ("d",""):"",    # "handkerchief"
    ("cq","kw"):"k",  # "aquire"
    ("ng","n"):"ng",
    ("kd","t"):"kt",
    ("nm","m"):"m", #government
    ("d","dʒ"):"d", #education
    ("x","kʃ"):"x",
    ("x","k"):"x", #excellent
    ("x","ʃ"):"x", #anxious
    ("l","ɝ"):"l", #colonel
    ("x","ɡʒ"):"x", #luxury
    ("s","tʃ"):"s" #tensions
}

In [169]:
def taylor_raw(word):
    consonants = "".join([taylor_convert[x] for x in process_word(word,ipa_dict, consonant_pairs)])
    vowels = "aeiouAEIOUɑæɪɔʊəɜɛːɒʌ"
    ipa = ipa_dict[word]
    punctuation_and_spaces = "ˈˌ. ,-'\";:!?()[]{}<>/@#%^&*~`"
    ipa =  ''.join([char for char in ipa if char not in punctuation_and_spaces])
    initial, final = False, False
    if ipa[0] in vowels:
        initial = True
    if len(consonants) > 0 and ipa[-1] in vowels:
        final = True
    if initial:
        consonants = "'" + consonants
    if final:
        consonants = consonants + "'"
    return consonants

In [170]:
taylor_endings = {
    'ing':(',',2),
    'ings':("_",3),
    'ble':("-b",2),
    'ful':("-f",2),
    'ly':(".",2),
    'ment':("-m",3),
    'ments':("-m",4),
    'ness':("-n",2),
    'rary':("-r",3),
    'self':("-s",3),
    'selvs':("-s",4),
    'ward':("-w",3),
    'ship':("-S",2),
    'ious':("I",1),
    'eous':("I",1),
    'uous':("I",1),
    'tion':("*",2),
    'sion':("*",2),
    'tions':("^",3),
    'sions':("^",3),
}

taylor_endings = {
    'ing':(',',2),
    'ings':("_",3),
    'ble':("b",2),
    'ful':("f",2),
    'ly':(".",2),
    'ment':("m",3),
    'ments':("m",4),
    'ness':("n",2),
    'rary':("r",3),
    'self':("s",3),
    'selvs':("s",4),
    'ward':("w",3),
    'ship':("S",2),
    'ious':("I",1),
    'eous':("I",1),
    'uous':("I",1),
    'tion':("*",2),
    'sion':("*",2),
    'tions':("^",3),
    'sions':("^",3),
}

taylor_briefs = {
    "be":"b",
    "by":"b",
    "been":"b",
    "do":"d",
    "did":"d",
    "of":"f",
    "off":"f",
    "if":"f",
    "god":"g",
    "give":"g",
    "go":"g",
    "have":"h",
    "he":"h",
    "know":"k",
    "known":"k",
    "no":"k",
    "lord":"l",
    "all":"l",
    "me":"m",
    "my":"m",
    "many":"m",
    "hand":"n",
    "and":"n",
    "an":"n",
    "in":"n",
    "peace":"p",
    "person":"p",
    "are":"r",
    "air":"r",
    "our":"r",
    "or":"r",
    "his":"s",
    "is":"s",
    "as":"s",
    "us":"s",
    "that":"t",
    "time":"t",
    "with":"w",
    "which":"w",
    "who":"w",
    "example":"x",
    "except":"x",
    "you":"y",
    "your":"y",
    "year":"y",
    "such":"C",
    "chance":"C",
    "shalt":"S",
    "shall":"S",
    "the":"T",
    "thee":"T",
    "they":"T",
    "conscious":"I",
    "judicious":"I"
}

def taylor_full(word):
    if word in taylor_briefs:
        return taylor_briefs[word]
    for ending in taylor_endings:
        if len(word) >= len(ending) and word[-len(ending):] == ending:
            symb, trim = taylor_endings[ending]
            return taylor_raw(word)[:-trim][:4] + symb
    return taylor_raw(word)

In [171]:
taylor = {}
failures = 0

for word in ipa_dict:
    try:
        taylor[word] = taylor_full(word)
    except:
        failures+=1

failures, "out of", len(ipa_dict), "failed."

(2283, 'out of', 125926, 'failed.')

In [172]:
code_statistics([x for x in one_grams_dict if x in taylor], taylor)

{'mean_binary_length': 12.003069219198045,
 'mean_binary_excess': 5.358130249730938,
 'mean_entropy_shift': 2.2383583238376072,
 'mean_transmission_cost': 10.244308800010597,
 'mean_transmission_excess': 3.5993698305434902,
 'mean_transmission_shift': 0.4795979046501593,
 'reconstruction_entropy': 0.7987322326991305,
 'reconstruction_error': 0.2218609124591735,
 'alphabet': "s r t n ' l d k p f m g b , * w h . T x C S ^ q c y _ I z"}

# Odell/Times Variant Approximation

This attempts to construct an allignment of the spelling and of the IPA sounds then construct the Taylor representation of that word. This is not a true implementation of Odell/Times Variant, but instead just allows for the the names of initial and final vowels to be kept.

In [173]:
# just kinda madeup lol
vowel_map = {
    "a":"a",
    "e":"e",
    "i":"i",
    "o":"o",
    "u":"u",
    "ɑ":"a",
    "æ":"a",
    "ɪ":"i",
    "ɔ":"o",
    "ʊ":"u",
    "ə":"e",
    "ɜ":"e",
    "ɛ":"e",
    "ɒ":"a",
    "ʌ":"a",
}

def taylor_plus_raw(word):
    consonants = "".join([taylor_convert[x] for x in process_word(word,ipa_dict, consonant_pairs)])
    vowels = "aeiouAEIOUɑæɪɔʊəɜɛːɒʌ"
    ipa = ipa_dict[word]
    punctuation_and_spaces = "ˈˌ. ,-'\";:!?()[]{}<>/@#%^&*~`"
    ipa =  ''.join([char for char in ipa if char not in punctuation_and_spaces])
    initial, final = False, False
    if ipa[0] in vowels:
        initial = vowel_map[ipa[0]]
        if word[0] in vowels:
            initial = word[0]
    if len(consonants) > 0 and ipa[-1] in vowels:
        final = vowel_map[ipa[-1]]
        if word[-1] in vowels:
            final = word[-1]
    if initial:
        consonants = initial + consonants
    if final:
        consonants = consonants + final
    return consonants

def taylor_plus_full(word):
    if word in taylor_briefs:
        return taylor_briefs[word]
    for ending in taylor_endings:
        if len(word) >= len(ending) and word[-len(ending):] == ending:
            symb, trim = taylor_endings[ending]
            return taylor_plus_raw(word)[:-trim][:4] + symb
    return taylor_plus_raw(word)

In [174]:
taylor_plus = {}
failures = 0

for word in ipa_dict:
    try:
        taylor_plus[word] = taylor_plus_full(word)
    except:
        failures+=1

failures, "out of", len(ipa_dict), "failed."

(2283, 'out of', 125926, 'failed.')

In [175]:
code_statistics([x for x in one_grams_dict if x in taylor_plus], taylor_plus)

{'mean_binary_length': 12.463657606792484,
 'mean_binary_excess': 5.818718637325378,
 'mean_entropy_shift': 2.698946711432047,
 'mean_transmission_cost': 10.845961925559017,
 'mean_transmission_excess': 4.201022956091911,
 'mean_transmission_shift': 1.0812510301985796,
 'reconstruction_entropy': 0.6857898653998239,
 'reconstruction_error': 0.1900756428174497,
 'alphabet': 's r t n l d k p f m g i b a , e * w h . T o x C S u ^ q c y _ I z'}

# Characterie

This uses a dictionary I made for [my webpage](https://characterie.neocities.org/).  This requires a custom alphabet and length measure that takes into account the strange way that characterie represents words.

In [176]:
words = {
    'a':['abound','about','accept','accuse','advance','air','again','age','all','almost','also','although','alter','am','amend','anger','anoint','apparel','appertain','appoint','arm','art','as','at','awe','away'], #added awe and away from the table of english words
    'b':['banish','bargain','bear','beast','beat','before','beg','begin','belly','bend','benefit','bestow','between','beware','by','because','bird','bishop','bite','blaze','blows','blood','blue','bless','bone','book','borrow','both','bottom','bread','break','breed','breast','bright','brittle','brother','bruise','burn','busy','but'], #blussy to blush by table guess
    'c':['call','can','captious','care','case','cave','cause','certain','challenge','change','christian','church','choose','kill','kind','circumstance','city','cloth','know','coin','color','command','comfort','common','compare','company','compel','continue','conceive','condition','contained','consider','confess','conscience','constant','convey','content','come','corner','corrupt','cover','council','count','cry','question','quit','compass','cut'], # caii to call and first cause to case by consulting Table.
    'd':['day','danger','deceive','declare','dedicate','dear','defend','delight','deprive','deputy','descend','desire','despise','destitute','destroy','diet','differ','dig','diligence','dissemble','distress','dizzy','do','doubt','draw','dream','dry','drink','drive','drop','due','double'], #dont to doubt by brachyography
    'e':['earth','edge','even','element','eloquence','enough','enter','enterprise','erect','err','escape','ever','example','except','exercise','expect','expert'], #every changed to ever due to overlap with all, second except changed to expect based on Bales
    'f':['face','faith','fair','fall','fare','far','fast','fat','fear','feast','feed','field','fetter','fight','fickle','fill','filthy','fine','find','fire','fish','flatter','flesh','fly','fling','flourish','follow','ford','force','forsake','fortune','foundation','fountain','free','friend','from','frown','fruit','furnish'], #third fair homophone changed to far.  Matches back table and Bales.
    'g':['gape','guard','garment','gather','gentle','guest','get','guide','given','go','god','good','gospel','glass','glory','grace','grass','grain','gravel','grave','great','grief','grove','grow'], #grassy to gravel by back table
    'h':['half','hand','hang','hard','heart','harvest','haste','have','halt','haunt','heal','he','head','help','herb','here','heat','hitherto','heaven','high','hill','history','hit','holy','hollow','honest','hope','how','hold','house','husband','hurt'], #hie to high and missing h to holy based on table
    'i':['yet','if','inheritance','enjoy','innocent','inquire','instrument','entertain','invent','you','join','young','judge','jewel'],
    'l':['labor','last','late','laugh','lean','learn','let','leather','leave','lie','liberality','life','light','like','limit','line','load','loose','love'], #ly to lie, lie to light by consulting back, removed louge for lack of matching backmatter word (all represented after other change)
    'm':['mad','make','man','manner','many','merchant','mark','marry','marvel','mass','master','matter','mean','measure','meet','mercy','merit','message','metal','mind','mine','mirth','mixed','mock','modesty','more','move','mouth','much','murmur'], #mast to master due to use with lord and maste going to rod in table (no master listed), many for money due to back and Bales
    'n':['nail','nature','necessary','neighbor','neither','net','nevertheless','nip','no','noble','nothing','nonetheless','now'],
    'o':['obey','office','offend','offer','oft','oh','omit','one','open','oppose','oppress','or','order','oath','other','over','overmuch','overtake','ought','own','our','out','outward'], 
    'p':['patient','parent','part','pass','peace','people','perfect','persuade','physique','place','plague','play','plain','plead','pledge','point','possible','power','pray','praise','preach','prejudice','prepare','present','pretend','prevail','prevent','prick','prince','promise','prophesy','proportion','prosper','prove','pulpit','punish','purge','purpose'],
    'r':['race','reign','rebuke','reach','recover','read','ready','region','rejoice','religion','remember','reap','repent','reason','resolve','rest','restore','reward','revenge','revile','rich','right','ripe','rob','rod','root','rough','rub','rule','rush'],
    's':['salute','save','scarce','school','slander','see','seed','seem','since','shine','ship','shoot','side','sink','sing','sit','skill','slip','smatter','smoke','sudden','soever','some','sore','sound','space','spare','spark','speak','spice','spit','spring','stay','start','step','steward','stone','strain','strong','study','stuff','stumble','substance','such','sweet','swell','surfeit','sun'], #shore to shoot to match back table, string to spring to also increase alignment
    't':['tame','taste','tear','temper','tempest','thank','that','then','thence','there','thither','thine','thing','think','this','thrive','tidings','till','time','together','tongue','touch','trade','treason','tree','tribute','triumph','trouble','true','turn'], # second tree changed to true (matches English Words)
    'w':['way','vain','wait','wear','warn','watch','water','weapon','weary','venture','very','virtue','vessel','weather','what','where','wherefore','which','whore','will','vine','wind','winter','violence','wife','visit','witness','wood','word','world','worship','worthy','up','uproar','wrinkle','write','use'] #wayne to way, vnrove to uproar based on back
}

particles = ['the', 'we', 'i', 'well', 'etc', 'be', 'fie', 'hence', 'they', 'myself', 'ourselves', 'so', 'so_as', 'and', 'in', 'of', 'to', 'a', 'for', 'with', 'it', 'it_is', 'it_self', 'as_it_were', 'that_is_to_say', 'that', 'least_that', 'thou', 'ye', 'self', 'ward', 'amen']

encode = {
    "0":"aa",
    "1":"ba",
    "2":"ca",
    "3":"da",
    "4":"ea",
    "5":"fa",
    "6":"ga",
    "7":"ha",
    "8":"ia",
    "9":"la"
}

num_encode = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ@#$%^&*()[]{}"

for letter in words:
    for i in range(len(words[letter])):
        encode[words[letter][i]] = num_encode[i] + letter

for i in range(len(particles)):
    encode["." + particles[i]] = num_encode[i] + "." 

In [177]:
with open('manual_curate.json') as f:
    characterie_raw = json.load(f)

def characterie_length(in_string):
    count = 0
    words = in_string.split(" ")
    for word in words:
        parts = word.split("-")
        count += len(parts)-1
        segments = parts[0].split(">")
        for segment in segments:
            if "." in segment:
                count += 1
            elif segment.isupper():
                count += len(segment)
            else:
                count += 2
    return count

parts_map = {
    "past":"-", #left dot
    "future":"+", #right dot
    "mark":"+",
    "er":":",
    "s":"+",
    "ing":"_",
    "not":"!",
    "ship":" As",
    "hood":" As",
}

def characterie_encode(in_word):
    in_string = characterie_raw[in_word]
    output = []
    words = in_string.split(" ")
    for word in words:
        parts = word.split("-")
        segments = parts[0].split(">")
        out_seg = []
        for segment in segments:
            if segment.isupper():
                out_seg.append(segment.lower())
            else:
                out_seg.append(encode[segment])
        output_word = "".join(out_seg)
        for i in range(1,len(parts)):
            output_word += parts_map[parts[i]]
        output.append(output_word)
    return " ".join(output)
    
characterie = {w:characterie_encode(w) for w in characterie_raw}
    
# # the letters, left dot, right dot, right double dot, under double dot, negation
# # the 48 foot/orientation combinations possible
# # the 32 particles (some also letters, and thus somewhat overcounted)
# characterie_alphabet = set()
# characterie_alphabet = characterie_alphabet | set("abcdefghilmnoprsztu1.:_!")
# characterie_alphabet = characterie_alphabet | set(f"o{i}" for i in range(48))
# characterie_alphabet = characterie_alphabet | set(f"p{i}" for i in range(32))

In [178]:
code_statistics([x for x in one_grams_dict if x in characterie], characterie)

{'mean_binary_length': 15.229091785638134,
 'mean_binary_excess': 9.756127863469656,
 'mean_entropy_shift': 6.8783592319593065,
 'mean_transmission_cost': 14.893585983536319,
 'mean_transmission_excess': 9.42062206136784,
 'mean_transmission_shift': 6.542853429857491,
 'reconstruction_entropy': 0.21794984827213726,
 'reconstruction_error': 0.0662984787750377,
 'alphabet': 'c s a p w f b m d r t e h i l o g 5 1 C n S B 6 2 8 7 3 0 A D I F - 9 E : G K   _ L H N . O M J T P Q 4 Z V R * X U @ Y + # $ % W ) ^ ] { & ( ['}

# Teeline

Using the Anki deck from [here](https://ankiweb.net/shared/info/310534731).

In [179]:
from ankipandas import Collection

col = Collection("collection.anki21")

In [180]:
teeline = {}

stuff_map = {'+':'',
 '+C':' C',
 '+CH':' CH',
 '+CHmn':' CHmn',
 '+CHr':' CHr',
 '+CHs':' CHs',
 '+Cd':' Cd',
 '+Ci':' Ci',
 '+Cs':' Cs',
 '+ING':' i',
 '+N':' N',
 '+ONG':' o',
 '+S':' S',
 '+Si':' Si',
 '+W':' W',
 '+and':' and',
 '+ang':' a',
 '+ange':' ae',
 '+c':' c',
 '+ch':' ch',
 '+ci':' ci',
 '+cl':' cl',
 '+cr':' cr',
 '+eng':' e',
 '+iN':' iN',
 '+ibble':' I',
 '+ing':' i',
 '+ingC':' iC',
 '+ingc':' ic',
 '+inge':' ie',
 '+ingle':' ile',
 '+ingr':' ir',
 '+l':' l',
 '+li':' li',
 '+ly':' ly',
 '+nch':' nch',
 '+ong':' o',
 '+ung':' u'}

for _,row in col.notes.iterrows():
    info = row['nflds']
    head = info[1].replace("&nbsp;","")
    tail = info[2].replace("&nbsp;","")
    
    if "<" in info[1]:
        continue

    output = tail.split(" ")
    for i in range(len(output)):
        part = output[i]
        if len(part) > 0 and part[0] == '+':
            output[i] = stuff_map[part]
        else:
            output[i] = " "+output[i]
    final = "".join(output)[1:]

    if len(final) == 0:
        continue
        
    for word in head.split(" "):
        teeline[word] = final.lower()

teeline['things'] = 'th is'
teeline['sung'] = 's u'
teeline['compelling'] = 'cmpl i' # teeline['compelling'] = 'CMPL i'
teeline['mangle'] = 'm al'
teeline['tongue'] = 't o'
teeline['tongs'] = 't os'

In [181]:
code_statistics([x for x in one_grams_dict if x in teeline], teeline)

{'mean_binary_length': 13.596477070553554,
 'mean_binary_excess': 8.630874974793834,
 'mean_entropy_shift': 5.8566119086553075,
 'mean_transmission_cost': 11.844900245462405,
 'mean_transmission_excess': 6.879298149702685,
 'mean_transmission_shift': 4.105035083564158,
 'reconstruction_entropy': 0.21836014216752608,
 'reconstruction_error': 0.054781492737337456,
 'alphabet': 's r t n c   i l d m p a f h b w e o g v u x y k q j z / 8 -'}

# Dearborn Speedwriting

A dictionary of the original speedwriting from [here](https://pastebin.com/Ca0aRexr)

In [182]:
with open("dearborn.txt", "r") as file:
    speedwriting_raw = file.readlines()

speedwriting = {}

for line in speedwriting_raw:
    splits = line.replace("\n","").split(" = ")
    if len(splits) != 2:
        continue

    word = splits[0]
    rep = splits[1]
    speedwriting[word.lower()] = rep

In [183]:
code_statistics([x for x in one_grams_dict if x in speedwriting], speedwriting)

{'mean_binary_length': 9.891620202359249,
 'mean_binary_excess': 4.734476718881333,
 'mean_entropy_shift': 1.9143101331526058,
 'mean_transmission_cost': 8.127170195054237,
 'mean_transmission_excess': 2.9700267115763213,
 'mean_transmission_shift': 0.149860125847594,
 'reconstruction_entropy': 0.6123703821986793,
 'reconstruction_error': 0.18216280866243562,
 'alphabet': "l s t k a n b p r m j d v - o f w , g i u / e h c y x S ; z K Z q ' T A N M L P W B H V G R F D O E U C X I J Q 2 3 Y"}

# Keyscript

Using a Keyscript dictionary extracted from the original PDF.

In [184]:
with open('keyscript.json') as f:
    keyscript_raw = json.load(f)

keyscript = {}
for word in keyscript_raw:
    keyscript[word.lower()] = keyscript_raw[word]

In [185]:
code_statistics([x for x in one_grams_dict if x in keyscript], keyscript)

{'mean_binary_length': 6.332050766621434,
 'mean_binary_excess': 2.061107371693759,
 'mean_entropy_shift': -0.5181941605179468,
 'mean_transmission_cost': 6.224301503986778,
 'mean_transmission_excess': 1.9533581090591037,
 'mean_transmission_shift': -0.6259434231526022,
 'reconstruction_entropy': 1.0871969584544927,
 'reconstruction_error': 0.27275805499274164,
 'alphabet': 's n r x c e i l k w t d j p a f m o h u y v z b q g 8 9'}

# QC-Line Base

In [186]:
# Pattern to remove vowels that are not the first or last letter, and double consonants
vowel_pattern = r"(?<!^)[aeiou](?!$)"
double_consonant_pattern = r"(?<=(.))\1"

def replace_qc(s): #this is approximate, but should be good enough to plot it
    # Replace 'c' followed by 'e', 'i', or 'y' with 's'
    s = re.sub(r"c(?=[eiy])", "s", s)
    # Replace remaining 'c's with 'k' and 'q's with 'k'
    s = re.sub(r"[cq]", "k", s)
    return s

# Apply the vowel removal pattern first, then the double consonant pattern
qc_line = {word:re.sub(vowel_pattern, '', re.sub(double_consonant_pattern, '', replace_qc(word))) for word in one_grams_dict}

In [187]:
code_statistics([x for x in one_grams_dict if x in qc_line], qc_line)

{'mean_binary_length': 15.39536827849569,
 'mean_binary_excess': 8.748415458722716,
 'mean_entropy_shift': 5.628221908026719,
 'mean_transmission_cost': 14.072462474995918,
 'mean_transmission_excess': 7.425509655222944,
 'mean_transmission_shift': 4.305316104526947,
 'reconstruction_entropy': 0.2327331929045948,
 'reconstruction_error': 0.0647610299624436,
 'alphabet': "s t n r l d k p g e m h y b f v a w i o x u j z ' é"}

# Yash

A rough approximation to Yash.

In [188]:
# THIS IS THE ONE THAT WORKS MOSTLY

# Remove punctuation and stress markers
def clean_word(word):
    punctuation_and_spaces = "ˈˌ. ,-'\";:!?()[]{}<>/@#%^&*~`"
    return ''.join([char for char in word if char not in punctuation_and_spaces])

# Recursive alignment function with debug statements
def find_allowable_pairs_recursive(str1, str2, allowable_pairs, debug=False):
    def helper(s1, s2, pairs, path):
        if not s1 and not s2:
            return path
        for pair in pairs:
            l1, l2 = len(pair[0]), len(pair[1])
            if s1[:l1] == pair[0] and s2[:l2] == pair[1]:
                if debug:
                    print(f"Matching: {pair} with {s1[:l1]} and {s2[:l2]}")
                result = helper(s1[l1:], s2[l2:], pairs, path + [pair])
                if result is not None:
                    return result
        if debug:
            print(f"Failed to match: {s1} with {s2}")
        return None

    return helper(str1, str2, allowable_pairs, [])

# Function to process the word and find alignment with debug statements
def process_word(word, ipa_representation, allowable_pairs, debug=False):
    cleaned_word = clean_word(word)
    cleaned_ipa = clean_word(ipa_representation)
    
    if debug:
        print(f"Processing '{word}' -> '{cleaned_word}'")
        print(f"IPA '{ipa_representation}' -> '{cleaned_ipa}'")
    
    alignment = find_allowable_pairs_recursive(cleaned_word, cleaned_ipa, allowable_pairs, debug=debug)
    
    return alignment

# Combined allowable pairs for both consonants and vowels
allowable_pairs = [
    # Consonant Combinations
    ("ch", "ʧ"), ("ch", "tʃ"), ("sh", "tʃ"), ("sh", "ʧ"), ("sh", "ʃ"),
    ("ch", "ʃ"), ("th", "θ"), ("th", "ð"), ("t", "tʃ"), ("g", "ʒ"),
    ("s", "z"), ("s", "ʒ"), ("s", "ʃ"), ("z", "ʒ"), ("z", "ʃ"), ("c", "ʃ"),
    ("ck", "k"), ("ps", "s"), ("q", "kw"), ("gh", "f"), ("ph", "f"),
    ("kn", "n"), ("gn", "n"), ("wr", "r"), ("wh", "w"), ("sc", "s"),
    ("c", "s"), ("ld", "d"), ("dd", "d"), ("ll", "l"), ("ll", "ɫ"),
    ("mm", "m"), ("nn", "n"), ("mn","m"), ("rr", "r"), ("pp", "p"), ("ff", "f"),
    ("tt", "t"), ("ss", "s"), ("ss", "z"), ("ss", "ʒ"), ("ss", "ʃ"),
    ("t", "ʒ"), ("t", "ʃ"), ("x", "gz"), ("x", "z"), ("zz", "ts"),
    ("z", "ts"), ("g", "dʒ"), ("g", "ʤ"), ("gg", "g"), ("gg", "ʒ"),
    ("gg", "dʒ"), ("gg", "ʤ"), ("ch", "k"), ("cc", "k"), 
    ("sl", "l"), ("q", "k"), ("l", "r"), 
    ("ct", "t"), ("dn", "n"),("ç", "s"),
    ("gm", "m"),("sc", "ʃ"), ("x", "ɡz"), ("gg", "ɡ"),
    ("cq", "kw"), ("ng", "n"), ("kd", "t"), ("nm", "m"), ("d", "dʒ"),
    ("x", "kʃ"), ("x", "k"), ("x", "ʃ"), ("l", "ɝ"), ("x", "ɡʒ"), ("s", "tʃ"),
    ("f", "v"), ("fth","θ"), ("cz","tʃ"), ("k","keɪ"),
    ("kn", "n"), ("gn", "n"), ("wr", "r"), ("mb", "m"),
    ("gu", "g"), ("que", "k"),("wh","hw"), ("g", "k"), ("c","tʃ"),("an","æm"),("th","tθ"),

    # Consonants
    ("c", "k"), ("t", "t"), ("d", "d"), ("d", "t"), ("t", "d"), ("n", "n"),
    ("s", "s"), ("l", "l"), ("l", "ɫ"), ("r", "r"), ("r", "ɹ"), ("r", "ɝ"),
    ("m", "m"), ("b", "b"), ("k", "k"), ("f", "f"), ("g", "g"), ("g", "ɡ"),
    ("v", "v"), ("x", "ks"), ("z", "z"), ("p", "p"), ("h", "h"), ("w", "w"), ("w", "hw"),
    ("y", "j"), ("j", "dʒ"), ("j", "ʤ"), ("j", "ʒ"), ("ng", "ŋ"), ("n", "ŋ"),

    # Vowel combinations
    ("ee","ɪ"),("ee","ə"),
    ("oo", "u"), ("oo", "ʊ"), ("ea", "i"), ("ee", "i"), 
    ("ai", "eɪ"), ("ay", "eɪ"), ("ei", "eɪ"), ("ey", "eɪ"), ("ai","ɛ"), ("ai","iɪ"),
    ("ie", "aɪ"), ("igh", "aɪ"),
    ("oa", "oʊ"), ("ow", "oʊ"), ("ow","aʊ"),
    ("ew", "ju"), ("ue", "ju"),
    ("au", "ɔ"), ("aw", "ɔ"), ("au","aʊ"), ('au', 'ə'), ("au","ɑ"),
    ("ou", "aʊ"), ("ou", "u"), ("ou", "ʊ"), ("ou","ɔ"),
    ("oi", "ɔɪ"), ("oy", "ɔɪ"), ("oo","ɪ"), ("oo","wɑ"), ("oo","wɔ"),
    ("ea","ɪ"),("ea","iə"),
    ("ay", "i"),("ay","ɛ"),
    ("eau","oʊ"),
    ("ui","wɪ"),
    ("ua","wə"),
    ("ua","weɪ"),("ua","wɪ"),
    ("oi","wɑ"), ("ui","wi"),
    
    # Vowels
    ("a", "æ"), ("a", "ə"), ("a", "ɑ"), ("a", "ɔ"), ("a", "eɪ"), ("a", "ɒ"), ("a","ɛ"), ("a","ɪ"), ("a","aɪ"), ("a", "ə"), 
    ("e", "ɛ"), ("e", "i"), ("e", "ə"), ("e", "ɪ"), ("e","æ"), ("e", "ɑ"), ("e","eɪ"),
    ("i", "ɪ"), ("i", "aɪ"), ("i", "i"), ("i", "ɜ"), ("i", "ə"), ("i","j"), ("i","jɪ"),
    ("o", "ɔ"), ("o", "ə"), ("o", "oʊ"), ("o", "ɒ"), ("o", "ɑ"), ("o", "u"), ("o","ʊ"), ("o","ɪ"), ("o", "wə"),
    ("u", "ʊ"), ("u", "ə"), ("u", "ju"), ("u", "ʌ"), ("u", "ɜ"), ("u", "u"), ("u", "ɪ"), ("u","j"), ("u","jʊ"), ("u","ɛ"), ("u","jəw"), ("u", "juw"), ("u","ɑ"), ("u","uw"),("u","ɔ"),
    ("y", "ɪ"), ("y", "aɪ"), ("y", "i"), ("y", "ə"),

    # stupid r/w stuff
    ("ar", "ɝ"),  ("er", "ɝ"),  ("or", "ɝ"),  ("yr", "ɝ"),  ("re", "ɝ"),  ("ir", "ɝ"),  ("ir", "aɪɝ"),
    ("ar", "ɹɝ"), ("er", "ɹɝ"), ("or", "ɹɝ"), ("yr", "ɹɝ"), ("re", "ɹɝ"), ("ir", "ɹɝ"), ("ir", "aɪɹɝ"),
    ("ar", "ɝɹ"), ("er", "ɝɹ"), ("or", "ɝɹ"), ("yr", "ɝɹ"), ("re", "ɝɹ"), ("ir", "ɝɹ"), ("ir", "aɪɝɹ"),
    ("u", "ɝ"), ("ew", "u"),
    
    # Silent Fallbacks and magic schwas
    ("t", ""), ("l", ""), ("ch", ""), ("s", ""), ("z", ""), ("r", ""), ("d", ""), ("n",""), ("k",""), ("p", ""),
    ("a", ""), ("e", ""), ("i", ""), ("o", ""), ("u", ""), ("gh", ""), ("b", ""), ("h", ""), ("w", ""), 
    ("", "ə"), 

    # Magic appearing "z" ??
    ("","z"),

    # Speaking the letters
    ("s","ɛs"), ("m","ɛm"), ("d","di"), ("c", "si")
]

In [189]:
def aligned(word):
    if word not in ipa_dict:
        return None
    return process_word(word, ipa_dict[word], allowable_pairs, debug=False)

In [190]:
yash_pair_mapping = {
    # Consonant Combinations
    ("ch", "ʧ"):'j', ("ch", "tʃ"):'j', ("sh", "tʃ"):'j', ("sh", "ʧ"):'j', ("sh", "ʃ"):'c',
    ("ch", "ʃ"):'c', ("th", "θ"):'y', ("th", "ð"):'y', ("t", "tʃ"):'t', ("g", "ʒ"):'g',
    ("s", "z"):'s', ("s", "ʒ"):'s', ("s", "ʃ"):'s', ("z", "ʒ"):'ts', ("z", "ʃ"):'ts', ("c", "ʃ"):'s',
    ("ck", "k"):'k', ("ps", "s"):'ps', ("q", "kw"):'kv', ("gh", "f"):'f', ("ph", "f"):'f',
    ("kn", "n"):'n', ("gn", "n"):'n', ("wr", "r"):'r', ("wh", "w"):'v', ("sc", "s"):'sc',
    ("c", "s"):'s', ("ld", "d"):'ld', ("dd", "d"):'d', ("ll", "l"):'l', ("ll", "ɫ"):'l',
    ("mm", "m"):'m', ("nn", "n"):'n', ("mn","m"):'mn', ("rr", "r"):'r', ("pp", "p"):'p', ("ff", "f"):'f',
    ("tt", "t"):'t', ("ss", "s"):'s', ("ss", "z"):'s', ("ss", "ʒ"):'s', ("ss", "ʃ"):'s',
    ("t", "ʒ"):'c', ("t", "ʃ"):'c', ("x", "gz"):'ks', ("x", "z"):'ks', ("zz", "ts"):'ts',
    ("z", "ts"):'ts', ("g", "dʒ"):'g', ("g", "ʤ"):'g', ("gg", "g"):'g', ("gg", "ʒ"):'g',
    ("gg", "dʒ"):'g', ("gg", "ʤ"):'g', ("ch", "k"):'k', ("cc", "k"):'k', 
    ("sl", "l"):'sl', ("q", "k"):'k', ("l", "r"):'l', 
    ("ct", "t"):'ct', ("dn", "n"):'dn',("ç", "s"):'s',
    ("gm", "m"):'gm',("sc", "ʃ"):'sc', ("x", "ɡz"):'ks', ("gg", "ɡ"):'g',
    ("cq", "kw"):'kv', ("ng", "n"):'ng', ("kd", "t"):'kd', ("nm", "m"):'nm', ("d", "dʒ"):'d',
    ("x", "kʃ"):'ks', ("x", "k"):'ks', ("x", "ʃ"):'ks', ("l", "ɝ"):'l', ("x", "ɡʒ"):'ks', ("s", "tʃ"):'s',
    ("f", "v"):'f', ("fth","θ"):'fy', ("cz","tʃ"):'cts', 
    ("kn", "n"):'kn', ("gn", "n"):'gn', ("wr", "r"):'r', ("mb", "m"):'mb',
    ("gu", "g"):'g', ("que", "k"):'k',("wh","hw"):'v', ("g", "k"):'g', ("c","tʃ"):'c',("an","æm"):'n',("th","tθ"):'y',

    # Consonants
    ("c", "k"):'k', ("t", "t"):'t', ("d", "d"):'d', ("d", "t"):'d', ("t", "d"):'t', ("n", "n"):'n',
    ("s", "s"):'s', ("l", "l"):'l', ("l", "ɫ"):'l', ("r", "r"):'r', ("r", "ɹ"):'r', ("r", "ɝ"):'r',
    ("m", "m"):'m', ("b", "b"):'b', ("k", "k"):'k', ("f", "f"):'f', ("g", "g"):'g', ("g", "ɡ"):'g',
    ("v", "v"):'v', ("x", "ks"):'ks', ("z", "z"):'ts', ("p", "p"):'p', ("h", "h"):'h', ("w", "w"):'v', ("w", "hw"):'v',
    ("y", "j"):'i', ("j", "dʒ"):'j', ("j", "ʤ"):'j', ("j", "ʒ"):'i', ("ng", "ŋ"):'q', ("n", "ŋ"):'n',

    # Vowel combinations
    ("ee","ɪ"):'i', ("ee","ə"):'',
    ("oo", "u"):'o', ("oo", "ʊ"):'o', ("ea", "i"):'i', ("ee", "i"):'i', 
    ("ai", "eɪ"):'a', ("ay", "eɪ"):'a', ("ei", "eɪ"):'e', ("ey", "eɪ"):'e', ("ai","ɛ"):'a', ("ai","iɪ"):'i',
    ("ie", "aɪ"):'i', ("igh", "aɪ"):'a',
    ("oa", "oʊ"):'o', ("ow", "oʊ"):'v', ("ow","aʊ"):'v',
    ("ew", "ju"):'v', ("ue", "ju"):'u',
    ("au", "ɔ"):'o', ("aw", "ɔ"):'v', ("au","aʊ"):'a', ('au', 'ə'):'a', ("au","ɑ"):'a',
    ("ou", "aʊ"):'o', ("ou", "u"):'u', ("ou", "ʊ"):'o', ("ou","ɔ"):'o',
    ("oi", "ɔɪ"):'i', ("oy", "ɔɪ"):'i', ("oo","ɪ"):'i', ("oo","wɑ"):'o', ("oo","wɔ"):'o',
    ("ea","ɪ"):'i', ("ea","iə"):'e',
    ("ay", "i"):'i', ("ay","ɛ"):'e',
    ("eau","oʊ"):'o',
    ("ui","wɪ"):'i',
    ("ua","wə"):'e',
    ("ua","weɪ"):'a',("ua","wɪ"):'i',
    ("oi","wɑ"):'o', ("ui","wi"):'u',
    
    # Vowels
    ("a", "æ"):'.', ("a", "ə"):'', ("a", "ɑ"):'.', ("a", "ɔ"):'.', ("a", "eɪ"):'.', ("a", "ɒ"):'.', ("a","ɛ"):'', ("a","ɪ"):'', ("a","aɪ"):'.', ("a", "ə"):'', 
    ("e", "ɛ"):'', ("e", "i"):'.', ("e", "ə"):'', ("e", "ɪ"):'', ("e","æ"):'.', ("e", "ɑ"):'.', ("e","eɪ"):'..',
    ("i", "ɪ"):'.', ("i", "aɪ"):'.', ("i", "i"):'.', ("i", "ɜ"):'', ("i", "ə"):'', ("i","j"):'.', ("i","jɪ"):'.',
    ("o", "ɔ"):'.', ("o", "ə"):'', ("o", "oʊ"):'.', ("o", "ɒ"):'.', ("o", "ɑ"):'.', ("o", "u"):'.', ("o","ʊ"):'.', ("o","ɪ"):'', ("o", "wə"):'.',
    ("u", "ʊ"):'.', ("u", "ə"):'', ("u", "ju"):'.', ("u", "ʌ"):'', ("u", "ɜ"):'', ("u", "u"):'.', ("u", "ɪ"):'', ("u","j"):'.', ("u","jʊ"):'.', ("u","ɛ"):'', ("u","jəw"):'.', ("u", "juw"):'.', ("u","ɑ"):'.', ("u","uw"):'.',("u","ɔ"):'.',
    ("y", "ɪ"):'.', ("y", "aɪ"):'.', ("y", "i"):'i', ("y", "ə"):'',

    # stupid r/w stuff
    ("ar", "ɝ"):'r',  ("er", "ɝ"):'r',  ("or", "ɝ"):'r',  ("yr", "ɝ"):'r',  ("re", "ɝ"):'r',  ("ir", "ɝ"):'r',  ("ir", "aɪɝ"):'r',
    ("ar", "ɹɝ"):'r', ("er", "ɹɝ"):'r', ("or", "ɹɝ"):'r', ("yr", "ɹɝ"):'r', ("re", "ɹɝ"):'r', ("ir", "ɹɝ"):'r', ("ir", "aɪɹɝ"):'r',
    ("ar", "ɝɹ"):'r', ("er", "ɝɹ"):'r', ("or", "ɝɹ"):'r', ("yr", "ɝɹ"):'r', ("re", "ɝɹ"):'r', ("ir", "ɝɹ"):'r', ("ir", "aɪɝɹ"):'r',
    ("u", "ɝ"):'', ("ew", "u"):'v',
    
    # Silent Fallbacks and magic schwas
    ("t", ""):'', ("l", ""):'', ("ch", ""):'', ("s", ""):'', ("z", ""):'', ("r", ""):'', ("d", ""):'', ("n",""):'', ("k",""):'', ("p", ""):'',
    ("a", ""):'', ("e", ""):'', ("i", ""):'', ("o", ""):'', ("u", ""):'', ("gh", ""):'', ("b", ""):'', ("h", ""):'', ("w", ""):'', 
    ("", "ə"):'', 

    # Magic appearing "z" ??
    ("","z"):'',

    # Speaking the letters
    ("s","ɛs"):'s', ("m","ɛm"):'m', ("d","di"):'d', ("c", "si"):'s', ("k","keɪ"):'k'
}

In [191]:
def yash_aligned(word, verbose = False):
    if word == "a":
        return "a"
    if word == "i":
        return "i"
    if word == "not":
        return "x"
    
    base = aligned(word)
    if base == None:
        return None

    if verbose: 
        print(base)

    mapped = "".join([yash_pair_mapping[p] for p in base])

    if verbose: 
        print(mapped)

    mapped = mapped.replace("nd","x").replace("nt","x") # X rule
    mapped = mapped.replace("st","z") # Z rule
    mapped = mapped.replace("rld","w").replace("rt","w").replace("tr","w").replace("rd","w").replace("ld","w").replace("dl","w").replace("lt","w").replace("td","w") # W rules

    if verbose:
        print(mapped)

    # not a listed rule, but he seems to almost always leave off the 
    if mapped[-2:] == "cn":
        mapped = mapped[:-1]

    if mapped[-3:] == "cns":
        mapped = mapped[:-2] + "s"

    mapped = mapped.replace(".","") # get rid of significant vowel markes

    return mapped

In [192]:
yash = {x:yash_aligned(x) for x in one_grams_dict if yash_aligned(x) != None}

In [193]:
for w in list(yash):
    if yash[w] == '':
        del yash[w]

In [194]:
code_statistics([x for x in one_grams_dict if x in yash], yash)

{'mean_binary_length': 11.047253230410739,
 'mean_binary_excess': 4.403937749275396,
 'mean_entropy_shift': 1.2842536453698834,
 'mean_transmission_cost': 10.616356864060966,
 'mean_transmission_excess': 3.973041382925624,
 'mean_transmission_shift': 0.8533572790201109,
 'reconstruction_entropy': 0.6985334759219873,
 'reconstruction_error': 0.18388474099533814,
 'alphabet': 's r k l n p d m t v w i x f b g c z q a o h y j e u'}

# Jeake Philosophical Transaction No. 487

[This](https://royalsocietypublishing.org/doi/epdf/10.1098/rstl.1748.0041) system is very simple, and related closely to an experimental system I had code for already.

In [195]:
def translate_jotter(string, rules, default = "."):
    temp = "".join(map(lambda x: rules.get(x,""), string))
    if len(temp) > 0:
        return temp
    else:
        return default

def compile_rules_jotter(merge_list):
    rule_dict = {}
    for equiv_class in merge_list:
        head = equiv_class[0]
        for letter in equiv_class:
            rule_dict[letter] = head
    return rule_dict

In [196]:
jeake_rule = ['dt','lr','mn','uvw','sxz','bfp','qkg','y','c'] # Jeake Philosophical Transaction No. 487
compiled_jeake = compile_rules_jotter(jeake_rule)
jeake = {w:translate_jotter(w,compiled_jeake) for w in one_grams_dict}

def replace_c(s): #this is approximate, but should be good enough to plot it
    # Replace 'c' followed by 'e', 'i', or 'y' with 's'
    s = re.sub(r"c(?=[eiy])", "s", s)
    # Replace remaining 'c's with 'k'
    s = re.sub(r"c", "g", s)
    return s

def combine_letters(s):
    # Use regex to find consecutive identical letters and replace them with a single capital letter
    result = re.sub(r"(.)\1+", lambda m: m.group(1).upper(), s)
    return result

jeake = {w:combine_letters(replace_c(jeake[w])) for w in jeake}

In [197]:
code_statistics([x for x in one_grams_dict if x in jeake], jeake)

{'mean_binary_length': 10.531745445747445,
 'mean_binary_excess': 3.8847926259744714,
 'mean_entropy_shift': 0.7645990752784737,
 'mean_transmission_cost': 8.696481737246566,
 'mean_transmission_excess': 2.0495289174735927,
 'mean_transmission_shift': -1.070664633222405,
 'reconstruction_entropy': 1.4819684205694112,
 'reconstruction_error': 0.35139657535538305,
 'alphabet': 'l d m s b u g q y L D M S B G U Q .'}

# A Readable Polyphonic Cipher
Not shorthand per se, but related, [here](https://digitalcommons.butler.edu/wordways/vol8/iss1/16/)

In [198]:
polyphonic_rule = ['e','txz','acq','ilb','ogj','npkv','ryw','sfm','hdu']
compiled_polyphonic = compile_rules_jotter(polyphonic_rule)
polyphonic = {w:translate_jotter(w,compiled_polyphonic) for w in one_grams_dict}

In [199]:
code_statistics([x for x in one_grams_dict if x in polyphonic], polyphonic)

{'mean_binary_length': 14.335438221763479,
 'mean_binary_excess': 7.688485401990505,
 'mean_entropy_shift': 4.568291851294507,
 'mean_transmission_cost': 14.368874723309242,
 'mean_transmission_excess': 7.721921903536268,
 'mean_transmission_shift': 4.60172835284027,
 'reconstruction_entropy': 0.08207522153529219,
 'reconstruction_error': 0.023091815341297117,
 'alphabet': 'i n a e s r h o t'}

# Pitman 2000

This uses the output from [this online translator](https://steno.tu-clausthal.de/Pitman.php) with `proof:7` which returns the underlying ascii information that it uses to feed into metafont to draw the proper Pitman form.  This online translator uses a machine-generated outline, and is known to not be particularly accurate, so this dot location should taken with a grain of salt.  This is also essentially unabbreviated Pitman (aside from some brief forms), so it is not fully representative of true practical performance.

In [200]:
pitman2k_raw = pd.read_csv("Pitman.txt",sep=r"\|\|",engine='python',names=['word','outline'],header=None)

pitman2k = {}
pitman2k_novowel = {}

pitman_translation_dictionary = {
    # initial positional vowels
    "{1}":["1"],
    "{2}":["2"],
    "{3}":["3"],
    # connecters (ignoring concatenation)
    "&":[],
    "/":["/"],
    " ":[" "],
    "=":["="],
    # basic consonants
    "(b)":["b"],
    "(p)":["p"],
    "(d)":["d"],
    "(t)":["t"],
    "(v)":["v"],
    "(f)":["f"],
    "(dh)":["dh"],
    "(th)":["th"],
    "(zh)":["zh"],
    "(sh)":["sh"],
    "(ng)":["ng"],
    "(n)":["n"],
    "(m)":["m"],
    "(l)":["l"],
    "(r)":["r"],
    "(w)":["w"],
    "(hw)":["hw"],
    "(y)":["y"],
    "(h)":["h"],
    "(s)":["s"],
    "(z)":["z"],
    "(ch)":["ch"],
    "(jh)":["jh"],
    "(k)":["k"],
    "(g)":["g"],
    "(_r)":["_r"],
    # combining consonants with l
    "(b,l)":["bl"],
    "(p,l)":["pl"],
    "(d,l)":["dl"],
    "(t,l)":["tl"],
    "(v,l)":["vl"],
    "(f,l)":["fl"],
    "(dh,l)":["dhl"],
    "(th,l)":["thl"],
    "(zh,l)":["zhl"],
    "(sh,l)":["shl"],
    "(ng,l)":["ngl"],
    "(n,l)":["nl"],
    "(m,l)":["ml"],
    "(l,l)":["ll"],
    "(r,l)":["rl"],
    "(w,l)":["wl"],
    "(hw,l)":["hwl"],
    "(y,l)":["yl"],
    "(h,l)":["hl"],
    "(s,l)":["sl"],
    "(z,l)":["zl"],
    "(ch,l)":["chl"],
    "(jh,l)":["jhl"],
    "(k,l)":["kl"],
    "(g,l)":["gl"],
    "(_r,l)":["_rl"],
    # combining consonants with r
    "(b,r)":["br"],
    "(p,r)":["pr"],
    "(d,r)":["dr"],
    "(t,r)":["tr"],
    "(v,r)":["vr"],
    "(f,r)":["fr"],
    "(dh,r)":["dhr"],
    "(th,r)":["thr"],
    "(zh,r)":["zhr"],
    "(sh,r)":["shr"],
    "(ng,r)":["ngr"],
    "(n,r)":["nr"],
    "(m,r)":["mr"],
    "(l,r)":["lr"],
    "(r,r)":["rr"],
    "(w,r)":["wr"],
    "(hw,r)":["hwr"],
    "(y,r)":["yr"],
    "(h,r)":["hr"],
    "(s,r)":["sr"],
    "(z,r)":["zr"],
    "(ch,r)":["chr"],
    "(jh,r)":["jhr"],
    "(k,r)":["kr"],
    "(g,r)":["gr"],
    "(_r,r)":["_rr"],
    # basic vowels
    "[a]":["a"],
    "[aa]":["aa"],
    "[o]":["o"],
    "[oo]":["oo"],
    "[e]":["e"],
    "[ei]":["ei"],
    "[uh]":["uh"],
    "[ou]":["ou"],
    "[i]":["i"],
    "[ii]":["ii"],
    "[u]":["u"],
    "[uu]":["uu"],
    "[ai]":["ai"],
    "[oi]":["oi"],
    "[ow]":["ow"],
    "[yuu]":["yuu"],
    # punctuation
    "(_period_)":["punc:."],
    # briefs
    "(_i_)":["brief:i"],
    "(_a_)":["brief:a"],
    "(_all_)":["brief:all"],
    "(_and_)":["brief:and"],
    "(_as_)":["brief:as"],
    "(_but_)":["brief:but"],
    "(_you_)":["brief:_u_"],
    "(_first_)":["brief:first"],
    "(_would_)":["brief:would"],
    "(_with_)":["brief:with"],
    "(_is_)":["brief:is"],
    "(_how_)":["brief:how"],
    "(_in_)":["brief:in"],
    "(_who_)":["brief:who"],
    "(_you_)":["brief:you"],
    "(_to_)":["brief:to"],
    "(_too_)":["brief:too"],
    "(_of_)":["brief:of"],
    "(_oh_)":["brief:oh"],
    "(_on_)":["brief:on"],
    "(_should_)":["brief:should"],
    "(_the_)":["brief:the"],
    "(_u_)":["brief:_u_"],
    # numbers
    "(_zero_)":["#0"],
    "(_one_)":["#1"],
    "(_two_)":["#2"],
    "(_three_)":["#3"],
    "(_four_)":["#4"],
    "(_five_)":["#5"],
    "(_six_)":["#6"],
    "(_seven_)":["#7"],
    "(_eight_)":["#8"],
    "(_nine_)":["#9"],
    # decorations (need to fix to attach to previous consonant?)
    ",_str":["dec:str"],
    ",st":["dec:st"],
    ",ses":["dec:ses"],
    ",sis":["dec:sis"],
    ",sais":["dec:sais"],
    ",s":["dec:s"],
    ",S":["dec:s"],
    ";n":["dec:n"],
    ";sishn":["dec:sishn"],
    ";shn":["dec:shn"],
    ";Shn":["dec:shn"],
    ";f":["dec:f"],
    ";v":["dec:v"],
    # prefix
    "^con":["prefix:con"],
    "^h":["prefix:h"],
    # suffix
    "~ing":["suffix:ing"],
    # sizes
    ":tr":["size:2"],
    ":t":["size:0.5"],
    ":dr":["size:2"],
    ":dhr":["size:2"],
    ":d":["size:0.5"],
    # unknown bits, might be translated wrong
    "[_ow]":["ow"],
    "[ow_]":["ow"],
    "[_yuu]":["yuu"],
    "[i_]":["i"],
    "[ii_]":["ii"],
    "[ai_]":["ai"],
    "[_ai]":["ai"],
    "(_v,r)":["vr"],
    "(_v,l)":["vl"],
    "(k,w)":["kw"],
    "(g,w)":["gw"],
    "[ai{(0u,-13u)}]":["ai"],
    "[ou{(5u,-5u)}]":["ou"],
    "[e{(10u,5u)}]":["e"],
    "[i{(2u,10u)}]":["i"],
    "[ai{(-2u,-25u)}]":["ai"],
    "[e{(5u,0u)}]":["e"],
    "[e{(-7u,7u)}]":["e"],
    "(_dh,r)":["dhr"],
    "(_l_)":["l"],
    "(_sh)":["sh"],
    "(_l)":["l"],
    "(l_)":["l"],
    "{_three_}":["3"],
    "(m_)":["m"],
    "(n_)":["n"],
    "(_m)":["m"],
    "(_n)":["n"],
    "[ou_]":["ou"],
    "(_zh)":["zh"],
    "[uu_]":["uu"]
}

pitman_vowels = {"a","aa","o","oo","e","ei","uh","ou","i","ii","u","uu","ai","oi","ow","yuu"}

def pitman_translate(outline, verbose = False, vowel_drop_percent = 0.0):
    pos = 0
    out = []
    
    while pos < len(outline):
        matched = False
        for head in pitman_translation_dictionary:
            if head == outline[pos:pos+len(head)]:
                out+=pitman_translation_dictionary[head]
                pos += len(head)
                matched = True
                break
        if not matched:
            if verbose:
                print(f"Failed in {outline} at {outline[pos:]}")
            return None
    vowel_dropped = []
    for x in out:
        if x not in pitman_vowels or random.random() > vowel_drop_percent:
            vowel_dropped.append(x)
    #print(out,vowel_dropped)
    return vowel_dropped

for _,line in pitman2k_raw.iterrows():
    translated = pitman_translate(line['outline'],verbose=True,vowel_drop_percent=0.0)
    translated_novowel = pitman_translate(line['outline'],verbose=True,vowel_drop_percent=1.0)
    if translated == None:
        #print(f"{line['word']:<10}{line['outline']:<20}{translated}")
        continue
    else:
        pitman2k[line['word']] = tuple(translated)
        pitman2k_novowel[line['word']] = tuple(translated_novowel)

print(len(pitman2k),len(pitman2k_raw))

999 999


In [201]:
[pitman2k[x] for x in pitman2k if pitman2k[x][0] in ['1','2','3']]

[('1', 'brief:all', 'l', 's'),
 ('1', 'brief:all', 'w', 'dec:s'),
 ('1', 'n'),
 ('3', 'd', 'dec:f'),
 ('3', 'dr', 'e', 'k', 'size:0.5'),
 ('1', 'd'),
 ('1', 'f'),
 ('1', 'd'),
 ('3', 'h', 'w', 'dec:s'),
 ('3', 'r'),
 ('3', 'f'),
 ('3', 'dec:s', 't'),
 ('1', 'jh'),
 ('1', 'mr'),
 ('3', 'r'),
 ('1', 'pr', 'size:0.5'),
 ('2', 'pr', 'h', 'a', 'p', 'dec:s'),
 ('3', 'p'),
 ('1', 'shr', 'size:0.5'),
 ('1', 'th'),
 ('1', 'dh', 'size:0.5'),
 ('3', 'l'),
 ('3', '_r')]

In [202]:
code_statistics([x for x in one_grams_dict if x in pitman2k], pitman2k)

{'mean_binary_length': 13.634455336659894,
 'mean_binary_excess': 9.008772205004034,
 'mean_entropy_shift': 6.377524380264968,
 'mean_transmission_cost': 14.005893355314635,
 'mean_transmission_excess': 9.380210223658775,
 'mean_transmission_shift': 6.748962398919709,
 'reconstruction_entropy': 0.06492057146871243,
 'reconstruction_error': 0.0188530058822729,
 'alphabet': 'dec:s e i size:0.5 l dec:n k m r n d ei a t o p ii ai ou _r uh f b w aa dec:st g uu ch v jh tr pr ow sh ng h size:2 u oo th yuu pl prefix:h gr s br bl 1 dr dec:v prefix:con kl dh vr fr dec:shn 3 hw y oi kw kr dhr tl dec:f fl gl brief:to brief:all zr thr suffix:ing dec:ses nr jhr brief:a brief:i brief:as _rr mr brief:too vl shl   #0 nl dec:str z brief:the brief:of brief:and brief:in brief:with brief:on brief:_u_ brief:is brief:but brief:would brief:who brief:how brief:first brief:should gw shr 2 dec:sishn brief:oh dl #1 #2 zhr dec:sais ml zh'}

In [203]:
code_statistics([x for x in one_grams_dict if x in pitman2k_novowel], pitman2k_novowel)

{'mean_binary_length': 10.107191714684433,
 'mean_binary_excess': 5.481508583028573,
 'mean_entropy_shift': 2.850260758289507,
 'mean_transmission_cost': 10.67896017671122,
 'mean_transmission_excess': 6.05327704505536,
 'mean_transmission_shift': 3.4220292203162943,
 'reconstruction_entropy': 0.41180440117056166,
 'reconstruction_error': 0.12174817795441739,
 'alphabet': 'dec:s size:0.5 l dec:n k m r n d t p _r f b w dec:st g ch v jh tr pr sh ng h size:2 th pl prefix:h gr s br bl 1 dr dec:v prefix:con kl dh vr fr dec:shn 3 hw y kw kr dhr tl dec:f fl gl brief:to brief:all zr thr suffix:ing dec:ses nr jhr brief:a brief:i brief:as _rr mr brief:too vl shl   #0 nl dec:str z brief:the brief:of brief:and brief:in brief:with brief:on brief:_u_ brief:is brief:but brief:would brief:who brief:how brief:first brief:should gw shr 2 dec:sishn brief:oh dl #1 #2 zhr dec:sais ml zh'}

In [204]:
from itertools import combinations

# Define the iterator class
class VowelFilterIterator:
    def __init__(self, outline, vowel_list):
        self.outline = outline
        self.vowel_list = set(vowel_list)
        self.vowel_indices = [i for i, ch in enumerate(outline) if ch in self.vowel_list]
        self.current_count = 0
        self.to_keep_combinations = []

    def __iter__(self):
        return self

    def __next__(self):
        # If all combinations for the current count are exhausted, increment count
        if not self.to_keep_combinations:
            if self.current_count > len(self.vowel_indices):
                raise StopIteration
            # Generate new combinations for the current count
            self.to_keep_combinations = list(combinations(self.vowel_indices, self.current_count))
            self.current_count += 1

        # Pick the next combination of vowel indices to keep
        keep_indices = set(self.to_keep_combinations.pop(0))
        sub_list = [ch for i, ch in enumerate(self.outline) if i not in self.vowel_indices or i in keep_indices]

        return sub_list

# Test case as per user's example
outline = list(pitman2k["populate"])
vowel_list = pitman_vowels

# Run the iterator and collect results for analysis
iterator = VowelFilterIterator(outline, vowel_list)
results = list(iterator)
results

[['p', 'p', 'l', 'size:0.5'],
 ['p', 'o', 'p', 'l', 'size:0.5'],
 ['p', 'p', 'yuu', 'l', 'size:0.5'],
 ['p', 'p', 'l', 'ei', 'size:0.5'],
 ['p', 'o', 'p', 'yuu', 'l', 'size:0.5'],
 ['p', 'o', 'p', 'l', 'ei', 'size:0.5'],
 ['p', 'p', 'yuu', 'l', 'ei', 'size:0.5'],
 ['p', 'o', 'p', 'yuu', 'l', 'ei', 'size:0.5']]

In [205]:
pitman_vowel_optimized = {}
used = set()

failures = 0
for word in one_grams_dict:
    if word not in pitman2k:
        continue
    
    for candidate in VowelFilterIterator(list(pitman2k[word]), pitman_vowels):
        test = tuple(candidate)
        if test not in used:
            pitman_vowel_optimized[word] = test
            used.add(test)
            break
    
    if word not in pitman_vowel_optimized:
        failures += 1
        pitman_vowel_optimized[word] = pitman2k[word]

failures

24

In [206]:
code_statistics([x for x in one_grams_dict if x in pitman_vowel_optimized], pitman_vowel_optimized)

{'mean_binary_length': 11.136021518086377,
 'mean_binary_excess': 6.510338386430517,
 'mean_entropy_shift': 3.8790905616914513,
 'mean_transmission_cost': 11.644002652258472,
 'mean_transmission_excess': 7.018319520602612,
 'mean_transmission_shift': 4.387071695863546,
 'reconstruction_entropy': 0.07683250655124771,
 'reconstruction_error': 0.022772203395222124,
 'alphabet': 'dec:s size:0.5 l dec:n k m r n d t p _r f b w ei dec:st ai ou g e ii ch v jh tr pr sh a ng h i size:2 o th pl prefix:h gr s br bl 1 uh dr dec:v prefix:con kl uu dh vr oo fr ow dec:shn 3 hw y kw kr u aa dhr yuu tl dec:f fl gl oi brief:to brief:all zr thr suffix:ing dec:ses nr jhr brief:a brief:i brief:as _rr mr brief:too vl shl   #0 nl dec:str z brief:the brief:of brief:and brief:in brief:with brief:on brief:_u_ brief:is brief:but brief:would brief:who brief:how brief:first brief:should gw shr 2 dec:sishn brief:oh dl #1 #2 zhr dec:sais ml zh'}

# Plotting

Now we plot!  I will also include an allowable region based upon using Fano's inequality to this problem. 

In [207]:
def word_list(dict):
    return [x for x in one_grams_dict if x in dict]

scores = []
scores.append(("Spelling",code_statistics(word_list(spelling), spelling),'top center'))
scores.append(("IPA",code_statistics(word_list(reduced_ipa), reduced_ipa),'top center'))
scores.append(("Carter</br></br>Briefhand",code_statistics(word_list(briefhand), briefhand),'top center'))
scores.append(("Dutton</br></br>Speedwords",code_statistics(word_list(speedwords), speedwords),'top center'))
scores.append(("Gregg</br></br>Anniversary",code_statistics(word_list(gregg_anniversary), gregg_anniversary, length_function = gregg_length, letter_making = gregg_letters),'top center'))
scores.append(("Gregg</br></br>Notehand",code_statistics(word_list(gregg_notehand), gregg_notehand, length_function = gregg_length, letter_making = gregg_letters),'middle left'))
scores.append(("Gregg</br></br>Simplified",code_statistics(word_list(gregg_simplified), gregg_simplified),'top center'))
scores.append(("bref",code_statistics(word_list(bref), bref),'top center'))
scores.append(("Yublin",code_statistics(word_list(yublin), yublin),'top center'))
scores.append(("Cut Spelng",code_statistics(word_list(cut_spelng), cut_spelng),'top center'))
scores.append(("Taylor",code_statistics(word_list(taylor), taylor),'top center'))
scores.append(("Taylor+",code_statistics(word_list(taylor_plus), taylor_plus),'top center'))
scores.append(("Characterie",code_statistics(word_list(characterie), characterie),'top center'))
scores.append(("Teeline",code_statistics(word_list(teeline), teeline),'bottom center'))
scores.append(("Keyscript",code_statistics(word_list(keyscript), keyscript),'top center'))
scores.append(("Dearborn</br></br>Speedwriting",code_statistics(word_list(speedwriting), speedwriting),'top center'))
scores.append(("QC-line",code_statistics(word_list(qc_line), qc_line),'middle right'))
scores.append(("Yash",code_statistics(word_list(yash), yash),'bottom center'))
scores.append(("Jeake</br></br>Ph. Tr. 487",code_statistics(word_list(jeake), jeake),'top center'))
scores.append(("Polyphonic</br></br>Cipher",code_statistics(word_list(polyphonic), polyphonic),'top right'))
scores.append(("Pitman</br></br>2000",code_statistics(word_list(pitman2k), pitman2k),'top center'))
scores.append(("Pitman 2000</br></br>(No Vowel)",code_statistics(word_list(pitman2k_novowel), pitman2k_novowel),'top center'))
scores.append(("Pitman 2000</br></br>(Optimal Vowel)",code_statistics(word_list(pitman_vowel_optimized), pitman_vowel_optimized),'bottom left'))

In [208]:
with open(f"system_scores_clean.json", "w") as f:
            f.write(json.dumps(scores))

In [209]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=np.linspace(-1.5,10,24), y=[max(0,-x/4) for x in np.linspace(-1.5,10,24)], mode='none', fill='tozeroy', name = "Excluded Region", fillcolor='#EF553B'))
fig.add_trace(go.Scatter(x=np.linspace(-1.5,10,24), y=[-1 for x in np.linspace(-1.5,10,24)], mode='none', fill='tozeroy', showlegend=False, fillcolor='#EF553B'))
fig.add_trace(go.Scatter(x=np.linspace(-1.5,10,24), y=[0 for x in np.linspace(-1.5,10,24)], mode='lines', fill='none', showlegend=False,line=dict(color='#EF553B')))

fig.add_trace(go.Scatter(x=[x[1]['mean_transmission_shift'] for x in scores],
                         y=[x[1]['reconstruction_error'] for x in scores],
                         text=[x[0] for x in scores], textposition=[x[2] for x in scores],
                         mode='markers+text', name="Shorthand Systems", line=dict(color='#636EFA')))
fig.update_layout(width=1280, height=1024, xaxis_range=[-1.5, 10], yaxis_range=[-0.01, 0.4],
                  xaxis_title="Average Outline Complexity Overhead (bits)",
                  yaxis_title="Reconstruction Error (probability)")
fig.show()

In [210]:
fig.write_image("comparison_graph.svg")

In [211]:
print(" ".join(word for word in list(one_grams_dict.keys())[0:2048]))

the of and to a in that is for i it was as not with he on you be his this by or are at her from she had but have an they we were one all can which their my if what do there when would him so more me will been out up about who has no into time other them your said could did then some these also like than its may only see new how two such over our first any just after back now through even people where well most between way because know should before many down very made life work those us here get use make being good much each both man right while used world same must long go years still day own does too under take another part state little however off three around think need never come might different again without god system going want hand social during place thought against something found high case say why away information head within let eyes number left old every great example since data came men set look children water university few face research chapter small things year based 